## Assemble reads *de novo* as another check to see if the Amaranthus hypochondriacus ingroup reference genome causes a bias. 
remove quinoa as an outgroup

In [1]:
# import packages into python
import ipyrad as ip
import ipyparallel as ipp
import ipyrad.analysis as ipa
import pandas as pd

# print the version of ipyrad you are running
print('ipyrad', ipa.__version__)

ipyrad 0.9.41


In [2]:
#Read in data and barcodes that are already created

COL_plates = "/moto/eaton/projects/RAW_DATA/Amaranthus/Amaranthus_COL_3RAD_run2_R*_.fastq.gz"
TUB_plate = "/moto/eaton/users/slh2181/tuberculatus_plate/Run_191713587/bcl2fastq2/Undetermined_S0_R*_001.fastq.gz"

BARCODES_I7_plate12 = "/moto/eaton/projects/RAW_DATA/Amaranthus/Amaranthus_COL_3RAD_run2_i7_barcodes.txt"
BARCODES_I7_tuberculatus = "/moto/eaton/users/slh2181/tuberculatus_plate/ipyrad/i7_barcodes_tub.txt"

INNER_BARCODES_plate1 = "/moto/eaton/users/slh2181/tuberculatus_plate/ipyrad/Amaranthus_UGA_3RAD_plate1_inline_barcodes.txt"
INNER_BARCODES_plate2 = "/moto/eaton/projects/RAW_DATA/Amaranthus/Amaranthus_COL_3RAD_plate2_inline_barcodes.txt"
INNER_BARCODES_plate3 = "/moto/eaton/users/slh2181/tuberculatus_plate/ipyrad/inner_barcodes_tub.txt"

# the reference Amaranthus genome
REF1 = "/moto/eaton/users/slh2181/reference/Ahypochondriacus_459_v2.0.fa"

# the reference Quinoa genome
REF2 = "/moto/eaton/projects/demux/quinoa.fna"

In [3]:
RAW_DATA_UGA =    "/moto/eaton/projects/RAW_DATA/Amaranthus/Amaranthus_UGA_3RAD_*.gz"
RAW_DATA_plate1 = "/moto/eaton/users/slh2181/tuberculatus_plate/ipyrad/demux_i7s_fastqs/plate1_*.gz"
RAW_DATA_plate2 = "/moto/eaton/users/slh2181/tuberculatus_plate/ipyrad/demux_i7s_fastqs/plate2_*.gz"
RAW_DATA_plate3 = "/moto/eaton/users/slh2181/tuberculatus_plate/ipyrad/demux_tub_fastqs/Amaranthus_*.gz"

In [4]:
# digest the beet genome and write as fastq data
dg2 = ipa.digest_genome(
    fasta="/moto/eaton/users/slh2181/reference/GCF_000511025.2_RefBeet-1.2.2_genomic.fna.gz",
    name="beet",
    workdir="/moto/eaton/users/slh2181/tuberculatus_plate/ipyrad/beet",
    re1="AGCTT",
    re2="CGG",
    paired=True,
    min_size=200,
    max_size=1000,
    ncopies=10,
    readlen=150,
    #nscaffolds=20,
)
dg2.run()

extracted 261106 reads


In [5]:
lib6 = ip.Assembly("beet")              #This is beet digested reference
lib6.params.sorted_fastq_path = "/moto/eaton/users/slh2181/tuberculatus_plate/ipyrad/beet/beet*.gz"
lib6.params.datatype = "pair3rad"
lib6.params.project_dir = "/moto/eaton/users/slh2181/tuberculatus_plate/ipyrad/beet_ref"
lib6.params.restriction_overhang = ("ATCGG", "TAGCTT")


lib6.run('1', force=True, auto=True)

New Assembly: beet
Parallel connection | t092: 24 cores
[####################] 100% 0:00:18 | loading reads        | s1 |


In [ ]:
# load each library from i7-demux'd fastq files
lib1 = ip.Assembly("UGA_plate1")          #This is UGA
lib1.params.raw_fastq_path = RAW_DATA_UGA
lib1.params.barcodes_path = INNER_BARCODES_plate1
lib1.params.datatype = "pair3rad"
lib1.params.project_dir = "/moto/eaton/users/slh2181/tuberculatus_plate/ipyrad/"
lib1.params.max_barcode_mismatch = 1
lib1.params.restriction_overhang = ("ATCGG", "TAGCTT")

lib2 = ip.Assembly("Columbia_plate1")     #This is Columbia plate 1
lib2.params.raw_fastq_path = RAW_DATA_plate1
lib2.params.barcodes_path = INNER_BARCODES_plate1
lib2.params.datatype = "pair3rad"
lib2.params.project_dir = "/moto/eaton/users/slh2181/tuberculatus_plate/ipyrad"
lib2.params.max_barcode_mismatch = 1
lib2.params.restriction_overhang = ("ATCGG", "TAGCTT")

lib3 = ip.Assembly("Columbia_contemp")     #This is Columbia (half) plate 2
lib3.params.raw_fastq_path = RAW_DATA_plate2
lib3.params.barcodes_path = INNER_BARCODES_plate2
lib3.params.datatype = "pair3rad"
lib3.params.project_dir = "/moto/eaton/users/slh2181/tuberculatus_plate/ipyrad"
lib3.params.max_barcode_mismatch = 1
lib3.params.restriction_overhang = ("ATCGG", "TAGCTT")

lib4 = ip.Assembly("tuberculatus")         #This is tuberculatus plate
lib4.params.raw_fastq_path = RAW_DATA_plate3
lib4.params.barcodes_path = INNER_BARCODES_plate3
lib4.params.datatype = "pair3rad"
lib4.params.project_dir = "/moto/eaton/users/slh2181/tuberculatus_plate/ipyrad"
lib4.params.max_barcode_mismatch = 1
lib4.params.restriction_overhang = ("ATCGG", "TAGCTT")

# run step 1 to sort/load fastq files 
lib1.run('1', force=True, auto=True)  #2 mins
lib2.run('1', force=True, auto=True)  #half hr
lib3.run('1', force=True, auto=True)  #10 mins
lib4.run('1', force=True, auto=True)  #1 hr

New Assembly: UGA_plate1
New Assembly: Columbia_plate1
New Assembly: Columbia_contemp
New Assembly: tuberculatus
Parallel connection | t092: 24 cores
[####################] 100% 0:01:17 | sorting reads        | s1 |
[####################] 100% 0:00:19 | writing/compressing  | s1 |
Parallel connection | t092: 24 cores
[                    ]   0% 0:07:59 | sorting reads        | s1 |

In [8]:
# load accession:species dataframe
spnames = pd.read_csv("/moto/eaton/users/slh2181/tuberculatus_plate/ipyrad/Amaranth_sample_names_tub.csv")

# subset only rows with species names not empty
spnames = spnames[spnames.Species.notna()]

# convert dataframe to a dictionary 
ndict = {spnames.Sample[i]: spnames.Species[i] for i in spnames.index}

# combine id and spp name into one long name
for key, value in ndict.items():
    ndict[key] = value + "_" + key

In [9]:
# create a merged assembly
merged = ip.merge(
    name="229_samp_beet", 
    assemblies=[lib1, lib2, lib3, lib4, lib6],
    rename_dict=ndict,
    )
merged.save()

New Assembly: 229_samp_beet


In [10]:
# set params on this assembly
merged.params.project_dir = "/moto/eaton/users/slh2181/tuberculatus_plate/ipyrad/"
merged.params.assembly_method = "reference"
merged.params.reference_sequence = REF1
merged.params.restriction_overhang = ("ATCGG", "TAGCTT")
merged.params.datatype = "pair3rad"
merged.params.phred_Qscore_offset = 43
merged.params.filter_adapters = 3
merged.params.mindepth_majrule = 3
merged.params.mindepth_statistical = 6
merged.params.max_barcode_mismatch = 1
merged.params.min_samples_locus = 10
merged.params.max_Ns_consens = 0.07
merged.params.max_Hs_consens = 0.07
merged.params.max_Indels_locus = 10
merged.params.output_formats = "psl"
merged.hackersonly.max_inner_mate_distance = 1000

merged.params

0   assembly_name               229_samp_beet                                
1   project_dir                 ./ipyrad                                     
2   raw_fastq_path              Merged: UGA_plate1, Columbia_plate1, Columbia_contemp, tuberculatus, beet
3   barcodes_path               Merged: UGA_plate1, Columbia_plate1, Columbia_contemp, tuberculatus, beet
4   sorted_fastq_path           Merged: UGA_plate1, Columbia_plate1, Columbia_contemp, tuberculatus, beet
5   assembly_method             reference                                    
6   reference_sequence          /moto/eaton/users/slh2181/reference/Ahypochondriacus_459_v2.0.fa
7   datatype                    pair3rad                                     
8   restriction_overhang        ('ATCGG', 'TAGCTT')                          
9   max_low_qual_bases          5                                            
10  phred_Qscore_offset         43                                           
11  mindepth_statistical        6      

In [11]:
merged.run("2", force=True, auto=True)

Parallel connection | t092: 24 cores
[####################] 100% 0:00:24 | concatenating inputs | s2 |
[####################] 100% 0:38:49 | processing reads     | s2 |


In [2]:
merged=ip.load_json("/moto/eaton/users/slh2181/tuberculatus_plate/ipyrad/229_samp_beet.json")

loading Assembly: 229_samp_beet
from saved path: /moto/eaton/users/slh2181/tuberculatus_plate/ipyrad/229_samp_beet.json


In [3]:
merged.run("34567", force=True, auto=True)

Parallel connection | t032: 24 cores
[####################] 100% 0:00:08 | indexing reference   | s3 |
[####################] 100% 0:03:38 | join unmerged pairs  | s3 |
[####################] 100% 0:02:06 | dereplicating        | s3 |
[####################] 100% 0:00:31 | splitting dereps     | s3 |
[####################] 100% 0:33:27 | mapping reads        | s3 |
[####################] 100% 0:46:33 | building clusters    | s3 |
[####################] 100% 0:00:20 | calc cluster stats   | s3 |
[####################] 100% 0:07:34 | inferring [H, E]     | s4 |
[####################] 100% 0:00:20 | calculating depths   | s5 |
[####################] 100% 0:00:40 | chunking clusters    | s5 |
[####################] 100% 0:46:45 | consens calling      | s5 |
[####################] 100% 0:05:23 | indexing alleles     | s5 |
[####################] 100% 0:00:42 | concatenating bams   | s6 |
[####################] 100% 0:00:14 | fetching regions     | s6 |
[####################] 100% 0:00:26 | b

In [4]:
merged = ip.load_json("/moto/eaton/users/slh2181/tuberculatus_plate/ipyrad/229_samp_beet.json")
denovo = merged.branch("229_samp_denovo")
denovo.params.assembly_method = "denovo"                
denovo.params.reference_sequence = ""
denovo.params

loading Assembly: 229_samp_beet
from saved path: /moto/eaton/users/slh2181/tuberculatus_plate/ipyrad/229_samp_beet.json


0   assembly_name               229_samp_denovo                              
1   project_dir                 ./ipyrad                                     
2   raw_fastq_path              Merged: UGA_plate1, Columbia_plate1, Columbia_contemp, tuberculatus, beet
3   barcodes_path               Merged: UGA_plate1, Columbia_plate1, Columbia_contemp, tuberculatus, beet
4   sorted_fastq_path           Merged: UGA_plate1, Columbia_plate1, Columbia_contemp, tuberculatus, beet
5   assembly_method             denovo                                       
6   reference_sequence                                                       
7   datatype                    pair3rad                                     
8   restriction_overhang        ('ATCGG', 'TAGCTT')                          
9   max_low_qual_bases          5                                            
10  phred_Qscore_offset         43                                           
11  mindepth_statistical        6                         

In [ ]:
denovo.run("34567", force=True, auto=True)

Parallel connection | t032: 24 cores
[####################] 100% 0:02:12 | join merged pairs    | s3 |
[####################] 100% 0:01:34 | join unmerged pairs  | s3 |
[####################] 100% 0:01:47 | dereplicating        | s3 |
[####################] 100% 0:52:24 | clustering/mapping   | s3 |
[####################] 100% 0:00:01 | building clusters    | s3 |
[####################] 100% 0:00:00 | chunking clusters    | s3 |
[####################] 100% 1:56:28 | aligning clusters    | s3 |
[####################] 100% 0:02:19 | concat clusters      | s3 |
[####################] 100% 0:00:21 | calc cluster stats   | s3 |
[####################] 100% 0:01:48 | inferring [H, E]     | s4 |
[####################] 100% 0:00:19 | calculating depths   | s5 |
[####################] 100% 0:00:20 | chunking clusters    | s5 |
[####################] 100% 0:28:28 | consens calling      | s5 |
[####################] 100% 0:02:01 | indexing alleles     | s5 |
[####################] 100% 0:01:06 | c

In [2]:
denovo= ip.load_json("/moto/eaton/users/slh2181/tuberculatus_plate/ipyrad/229_samp_denovo.json")
denovo.params

loading Assembly: 229_samp_denovo
from saved path: /moto/eaton/users/slh2181/tuberculatus_plate/ipyrad/229_samp_denovo.json


0   assembly_name               229_samp_denovo                              
1   project_dir                 ./ipyrad                                     
2   raw_fastq_path              Merged: UGA_plate1, Columbia_plate1, Columbia_contemp, tuberculatus, beet
3   barcodes_path               Merged: UGA_plate1, Columbia_plate1, Columbia_contemp, tuberculatus, beet
4   sorted_fastq_path           Merged: UGA_plate1, Columbia_plate1, Columbia_contemp, tuberculatus, beet
5   assembly_method             denovo                                       
6   reference_sequence                                                       
7   datatype                    pair3rad                                     
8   restriction_overhang        ('ATCGG', 'TAGCTT')                          
9   max_low_qual_bases          5                                            
10  phred_Qscore_offset         43                                           
11  mindepth_statistical        6                         

In [3]:
denovo.run("67", force=True, auto=True)

Parallel connection | t103: 24 cores
[####################] 100% 0:01:25 | concatenating inputs | s6 |
[###########         ]  56% 2:56:26 | clustering across    | s6 |

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[####################] 100% 5:27:21 | clustering across    | s6 |
[####################] 100% 0:00:59 | building clusters    | s6 |
[####################] 100% 0:20:43 | aligning clusters    | s6 |
[####################] 100% 0:00:24 | applying filters     | s7 |
[####################] 100% 0:05:12 | building arrays      | s7 |
[####################] 100% 0:03:14 | writing conversions  | s7 |


### drop samples with the lowest # loci

In [14]:
merged.stats_dfs.s7_samples.sample_coverage < 5000

reference                    False
acanthochiton_SLH_AL_0001    False
acanthochiton_SLH_AL_0002    False
acutilobus_SLH_AL_0003       False
acutilobus_SLH_AL_0004       False
                             ...  
viridis_SLH_AL_3047          False
viridis_SLH_AL_3062          False
watsonii_SLH_AL_3065         False
wrightii_SLH_AL_3066         False
wrightii_SLH_AL_3067         False
Name: sample_coverage, Length: 231, dtype: bool

In [19]:
mask = merged.stats_dfs.s7_samples.sample_coverage < 1000
lose = merged.stats_dfs.s7_samples[mask].index.tolist()
lose

['cannabinus_SLH_AL_0042',
 'unknown_SLH_AL_0055-contemp',
 'unknown_SLH_AL_0062-contemp']

In [20]:
mask = merged.stats_dfs.s7_samples.sample_coverage > 1000
keep = merged.stats_dfs.s7_samples[mask].index.tolist()
keep

['reference',
 'acanthochiton_SLH_AL_0001',
 'acanthochiton_SLH_AL_0002',
 'acutilobus_SLH_AL_0003',
 'acutilobus_SLH_AL_0004',
 'albus_SLH_AL_0006',
 'albus_SLH_AL_0009',
 'albus_SLH_AL_0010',
 'arenicola_SLH_AL_0012',
 'arenicola_SLH_AL_0013',
 'arenicola_SLH_AL_0015',
 'arenicola_SLH_AL_0018',
 'asplundii_SLH_AL_0019',
 'australis_SLH_AL_0020',
 'australis_SLH_AL_0021',
 'beet',
 'blitoides_SLH_AL_0023',
 'blitoides_SLH_AL_0028',
 'blitum-blitum_SLH_AL_0029',
 'blitum-oleraceus_SLH_AL_0034',
 'blitum-pseudogracilis_SLH_AL_0037',
 'californicus_SLH_AL_0039',
 'cannabinus_SLH_AL_0040',
 'cannabinus_SLH_AL_0041',
 'caudatus_SLH_AL_0102',
 'caudatus_SLH_AL_0110',
 'caudatus_SLH_AL_0116',
 'caudatus_SLH_AL_0322',
 'caudatus_SLH_AL_0540',
 'crassipes_SLH_AL_0599',
 'crassipes_SLH_AL_0600',
 'cruentus_SLH_AL_0679',
 'cruentus_SLH_AL_0699',
 'cruentus_SLH_AL_0728',
 'cruentus_SLH_AL_0804',
 'cruentus_SLH_AL_0832',
 'deflexus_SLH_AL_0951',
 'deflexus_SLH_AL_0952',
 'deflexus_SLH_AL_0953',
 '

In [21]:
new = merged.branch("Good_samples_beet2", subsamples=keep)
new.params

0   assembly_name               Good_samples_beet2                           
1   project_dir                 ./ipyrad                                     
2   raw_fastq_path              Merged: UGA_plate1, Columbia_plate1, Columbia_contemp, tuberculatus, quinoa, beet
3   barcodes_path               Merged: UGA_plate1, Columbia_plate1, Columbia_contemp, tuberculatus, quinoa, beet
4   sorted_fastq_path           Merged: UGA_plate1, Columbia_plate1, Columbia_contemp, tuberculatus, quinoa, beet
5   assembly_method             reference                                    
6   reference_sequence          /moto/eaton/users/slh2181/reference/Ahypochondriacus_459_v2.0.fa
7   datatype                    pair3rad                                     
8   restriction_overhang        ('ATCGG', 'TAGCTT')                          
9   max_low_qual_bases          5                                            
10  phred_Qscore_offset         43                                           
11  mindepth_st

In [22]:
new.run("7", force=True, ipyclient=ipyclient)

Parallel connection | t103: 24 cores
[####################] 100% 0:00:35 | applying filters     | s7 |
[####################] 100% 0:22:59 | building arrays      | s7 |
[####################] 100% 0:09:26 | writing conversions  | s7 |
